In [ ]:
# 二階booking.com
# 台北市-四人(只要換網址就可以變成二人)

from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import pymongo
import time
import random
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.common.exceptions import TimeoutException, WebDriverException, NoSuchElementException, StaleElementReferenceException

client = pymongo.MongoClient("mongodb+srv://帳號:密碼@mongodb連線設定")
db = client.資料庫名稱
collection = db.資料表名稱

def choose_checkin():
    driver.find_elements_by_css_selector('button[class="d47738b911 fb1847d86a"]')[0].click()
    time.sleep(1)
    checkin_date = driver.find_elements_by_css_selector('td[class="b21c1c6c83 e505d9d049"]')[1].get_attribute("innerText")
    checkin_month = driver.find_elements_by_css_selector('div[class="ab0d1629e5"]')[0].get_attribute("innerText")[:2]
    time.sleep(1)
    # 選日期
    driver.find_elements_by_css_selector('td[class="b21c1c6c83 e505d9d049"]')[1].click()
    # 確認
    driver.find_element_by_css_selector('button[class="fc63351294 a822bdf511 d4b6b7a9e7 f7db01295e af18dbd5a4 f4605622ad c827b27356"]').click()
    return checkin_month, checkin_date

desired_capabilities = DesiredCapabilities.CHROME # 調整selenium讀取方式
desired_capabilities["pageLoadStrategy"] = "none" # 直接使用DOM樹，進行操作
driver = webdriver.Chrome("C:\\Users\\Lihome\\chromedriver.exe")
booking = "https://www.booking.com/searchresults.zh-tw.html?ss=%E5%8F%B0%E5%8C%97&sb_travel_purpose=business&ssne=%E5%8F%B0%E5%8C%97&ssne_untouched=%E5%8F%B0%E5%8C%97&label=booking-name-yefrPbbyS*FIINHgyCnmNgS267725091255%3Apl%3Ata%3Ap1%3Ap22%2C563%2C000%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp9040379%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9YfqnDqqG8nt1O4nYvDr1lms&sid=cf35be32f813fdd7dabae80fcc85b1a3&aid=376396&lang=zh-tw&sb=1&src_elem=sb&src=searchresults&dest_id=-2637882&dest_type=city&checkin=2022-06-04&checkout=2022-06-05&group_adults=2&no_rooms=1&group_children=0"
driver.get(booking)

checkin_month = "6"
checkin_date = "4"
taipei_dist = ["松山", "信義", "大安", "中山", "中正", "大同", "萬華", "文山", "南港", "內湖", "士林", "北投"]

while True:
    if checkin_month == "6 " and checkin_date == "11":
        break
    else:
        # 找一頁的筆數
        data_count = len(driver.find_elements_by_css_selector('div[class="d20f4628d0"]'))
        time.sleep(3)
        # 找最後一頁的數字
        last_page = driver.find_elements_by_css_selector('li[class="f32a99c8d1"]')[-1].get_attribute("innerText")
        
        # 開始抓資料
        for x in range(len(last_page)):
            for hotel in range(data_count):
                # 地址，確認是不是台北的行政區
                try:
                    address = driver.find_elements_by_css_selector('span[data-testid="address"]').[hotel].get_attribute("innerText")
                    if address[0:2] not in taipei_dist:
                        continue
                    else:
                        address = address.replace(" ","")
                        if address[2] == ",":      # 判斷鄉鎮市區 沒區字
                            address = address[:2] + "區"
                        elif address[2] == "區":
                            address = address[:3]
                        elif address[:3] == "西門町":    # 判斷鄉鎮市區
                            address = address.replace("西門町","萬華區")
                except:
                    continue
                
#                 # 其他縣市需要這樣設定
#                 if address[1] == "區":     # 判斷鄉鎮市區
#                     address = address[:2]
#                 elif address == "台中":
#                     continue
#                 elif address[2] == "區":
#                     address = address[:3]
    
                # 飯店名稱
                try:
                    if driver.find_elements_by_css_selector('div[class="fcab3ed991 a23c043802"]')[hotel].get_attribute("innerText") == "是美精品飯店":
                        continue
                    else:
                        name = driver.find_elements_by_css_selector('div[class="fcab3ed991 a23c043802"]')[hotel].get_attribute("innerText")
                        print(name)
                except:
                    continue
                    
                # 網址
                hotel_link = driver.find_elements_by_css_selector('a[class="e13098a59f"]')[hotel].get_attribute("href")
                
                # 入住日期
                if len(checkin_month)<2:
                    if len(checkin_date)<2:
                        day = "2022-0%s-0%s"%(checkin_month, checkin_date)
                    else:
                        day = "2022-0%s-%s"%(checkin_month, checkin_date)
                else:
                    if len(checkin_date)<2:
                        day = "2022-%s-0%s"%(checkin_month, checkin_date)
                    else:
                        day = "2022-%s-%s"%(checkin_month, checkin_date)
                # 房型
                bed_num = driver.find_elements_by_css_selector('div[class="d22a7c133b"]')[hotel].get_attribute("innerText")
                if "2 張單人床" in bed_num:
                    roomtype = driver.find_elements_by_css_selector('span[class="df597226dd"]')[hotel].get_attribute("innerText")
                    roomtype = roomtype+"(2張床)"
                else:
                    roomtype = driver.find_elements_by_css_selector('span[class="df597226dd"]')[hotel].get_attribute("innerText")
                
                # 價格
                price = driver.find_elements_by_css_selector('span[class="fcab3ed991 bd73d13072"]')[hotel]
                price = price.text.replace("TWD ","")
                price = price.replace(",","")
                price = int(price)
                
                # 二階
                driver.find_elements_by_css_selector('a[class="fc63351294 a822bdf511 d4b6b7a9e7 f7db01295e f4605622ad b2f0d6a80e"]')[hotel].is_display()
                enter = driver.find_elements_by_css_selector('a[class="fc63351294 a822bdf511 d4b6b7a9e7 f7db01295e f4605622ad b2f0d6a80e"]')[hotel]
                enter.click()
                num = driver.window_handles
                driver.switch_to.window(num[-1])
                time.sleep(2)
                driver.excute_script("window.stop();")    # 網頁停止loading
                time.sleep(3)
                
                # 地圖連結
                try:
                    map_link = driver.find_element_by_css_selector('a[id="hotel_address"]').get_attribute("href")
                except:
                    driver.close()
                    driver.switch_to.window(num[0])
                    continue
                
                # 停車資訊
                try:
                    car_park = driver.find_elements_by_css_selector('div[class="bui-spacer--medium hotel-facilities-group__policy"]')[1].get_attribute("innerText")
                    if car_park == "住宿方于客房提供WiFi（免費）。":
                        car_park = driver.find_elements_by_css_selector('div[class="bui-spacer--medium hotel-facilities-group__policy"]')[2].get_sttribute("innerText")
                        if car_park[:5] == "無法預約：":
                            car_park_state = "Yes"
                            car_park = car_park[6:-1]
                            
                        elif car_park[:5] == "不需預約：":
                            car_park_state = "Yes"
                            car_park = car_park[6:-1]
                            
                        elif car_park[:4] == "需預約：":
                            car_park_state = "Yes"
                            car_park = car_park[5:-1]
                            
                        elif car_park == "無停車設施。":
                            car_park_state = "No"
                            car_park = car_park[:-1]
                            
                        elif car_park == "提供停車相關設施":
                            car_park_state = "Yes"
                        
                    elif car_park[:5] == "無法預約：":
                        car_park_state = "Yes"
                        car_park = car_park[6:-1]
                        
                    elif car_park[:5] == "不需預約：":
                        car_park_state = "Yes"
                        car_park = car_park[6:-1]
                        
                    elif car_park[:4] == "需預約：":
                        car_park_state = "Yes"
                        car_park = car_park[5:-1]
                        
                    elif car_park == "提供停車相關設施":
                        car_park_state = "Yes"
                        
                    elif car_park == "無停車設施。":
                        car_park_state = "No"
                        car_park = car_park[:-1]
                        
                except:
                    driver.close()
                    driver.switch_to.window(num[0])
                    continue
                    
                # 評分
                try:
                    location = driver.find_elements_by_css_selector('span[class="c-score-bar__score"]')[1].get_attribute("innerText")
                    location = float(location)
                    cleanliness = driver.find_elements_by_css_selector('span[class="c-score-bar__score"]')[2].get_attribute("innerText")
                    cleanliness = float(cleanliness)
                    comfort = driver.find_elements_by_css_selector('span[class="c-score-bar__score"]')[4].get_attribute("innerText")
                    comfort = float(comfort)
                except:
                    driver.close()
                    driver.switch_to.window(num[0])
                    continue
                    
                data = {
                    "Name":name,
                    "District":address,
                    "Link":hotel_link,
                    "Checkin_day":day,
                    "Roomtype":roomtype,
                    "Price":price,
                    "Map_link":map_link,
                    "Car_Park_State":car_park_state,
                    "Car_Park":car_park,
                    "Location":location,
                    "Cleanliness":cleanliness,
                    "Comfort":comfort
                }
                result = collection.insert_one(date)
                
                time.sleep(2)
                driver.close()
                driver.switch_to.window(num[0])
                
            # 下一頁
            try:
                last_page_button = driver.find_element_by_css_selector('li[class="f32a99c8d1 ebd02eda9e"]').get_attribute("innerText")
                if last_page_button == last_page:
                    continue
                else:
                    next_page = driver.find_element_by_css_selector('button[aria-label="下一頁"]').click()
                    time.sleep(random.randint(5,10))
                    print(last_page_button)   # 印出已完成頁數
                    
            except:
                print("不知道")
                
        checkin_month, checkin_date = choose_checkin()
        
print("結束")